In [18]:

import models
import utils
import data_processing
from constants import (
    VALUE_COLUMN,
    UNIT_RATE_COLUMN,
    QUANTITY_COLUMN,
    GROSS_WEIGHT_COLUMN,
    SHIP_COUNT_COLUMN,
    PORT_COUNT_COLUMN,
    FILL_METHOD,
    TARGET_COLUMN
)
import pandas as pd
import constants
import sys
from datetime import datetime
from build_data import get_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

In [19]:
COMMODITY = "magnesium"
SPIKES_WINDOW_SIZE = 20

VOLZA_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}.csv"
PRICE_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}_prices.csv"

In [20]:
data = get_data(VOLZA_FILE_PATH, PRICE_FILE_PATH, SPIKES_WINDOW_SIZE)

Building data...
Detecting spikes... 20


/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)


In [21]:
data

,Date,Value,Std. Quantity (KG),Gross Weight,Std. Unit Rate ($/KG),ship_count,popular_port,popular_port_count,Price,Brent Oil Value,WTI Oil Value,spikes
0,2020-01-01,3.716627e+07,1.451213e+07,275000.0,8.582012,NaN,NaN,NaN,NaN,67.77,61.14,0
1,2020-01-02,1.125000e+05,5.000000e+04,50080.0,2.250000,NaN,NaN,NaN,14150.0,67.05,61.17,0
2,2020-01-03,8.744076e+05,1.001723e+05,25040.0,768.449153,NaN,NaN,NaN,14100.0,69.08,63.00,0
3,2020-01-04,2.950597e+05,1.350000e+05,0.0,2.188610,NaN,NaN,NaN,14100.0,69.08,63.00,0
4,2020-01-05,4.420000e+04,2.000000e+04,0.0,2.210000,NaN,NaN,NaN,14100.0,69.08,63.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
964,2022-12-27,9.624998e+04,2.500000e+04,25040.0,3.849999,27245.0,SGSIN,9195.0,21700.0,82.45,79.45,0
965,2022-12-28,1.122651e+06,2.869969e+05,80306.0,5.656118,26021.0,SGSIN,9233.0,21650.0,81.70,78.89,0
966,2022-12-29,1.303649e+06,3.568780e+05,0.0,3.490332,24039.0,SGSIN,8861.0,21650.0,80.96,78.43,0
967,2022-12-30,9.095473e+05,2.128420e+05,44080.0,3.949859,19952.0,SGSIN,8118.0,21650.0,82.82,80.16,0


In [9]:
# Trying out different window sizes
SPIKE_WINDOW_SIZES = [10, 20, 30, 40, 60, 80, 100]
results_dfs = []

for window_size in SPIKE_WINDOW_SIZES:
    SPIKES_WINDOW_SIZE = window_size
    aggregated_df = get_data(VOLZA_FILE_PATH, PRICE_FILE_PATH, SPIKES_WINDOW_SIZE)
    VOLZA_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN]
    X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_FEATURES, 'spikes')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_test = data_processing.scale_features(X_train, X_test)

    X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
    X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)
    
    print(f'X train shape: {X_train.shape}')

    output_file_path = f'{COMMODITY}/volza/results_{window_size}.csv'
    pred_file_path = f'{COMMODITY}/volza/predictions/{window_size}'
    print(pred_file_path)
    results_df = models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path, pred_file_path)
    results_df['Window Size'] = window_size
    results_dfs.append(results_df)
    

Building data...
Detecting spikes... 10
X train shape: (766, 10, 4)
magnesium/volza/predictions/10


/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)


6/6 [==============================] - 0s 4ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_250_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_250_layers_predictions.npy
6/6 [==============================] - 0s 6ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_200_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_200_layers_predictions.npy
6/6 [==============================] - 0s 7ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_100_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_100_layers_predictions.npy
6/6 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/10/LSTM_50_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/10/LSTM_50_layers_predictions.npy
6/6 [==============================] - 0s 2ms/

/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)


6/6 [==============================] - 0s 9ms/step
Predictions saved to CSV file: magnesium/volza/predictions/20/LSTM_250_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/20/LSTM_250_layers_predictions.npy
6/6 [==============================] - 0s 7ms/step
Predictions saved to CSV file: magnesium/volza/predictions/20/LSTM_200_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/20/LSTM_200_layers_predictions.npy
6/6 [==============================] - 0s 4ms/step
Predictions saved to CSV file: magnesium/volza/predictions/20/LSTM_100_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/20/LSTM_100_layers_predictions.npy
6/6 [==============================] - 0s 4ms/step
Predictions saved to CSV file: magnesium/volza/predictions/20/LSTM_50_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/20/LSTM_50_layers_predictions.npy
6/6 [==============================] - 0s 2ms/

...

3/3 [==============================] - 0s 19ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/LSTM_100_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/LSTM_100_layers_predictions.npy
3/3 [==============================] - 0s 7ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/LSTM_50_layers_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/LSTM_50_layers_predictions.npy
3/3 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_Attention_32_filters_7_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_Attention_32_filters_7_kernels_predictions.npy
3/3 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_Attention_32_filters_5_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_Attentio

/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

3/3 [==============================] - 0s 4ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/RNN_50_units_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/RNN_50_units_predictions.npy


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

3/3 [==============================] - 0s 3ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_32_filters_7_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_32_filters_7_kernels_predictions.npy
3/3 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_32_filters_5_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_32_filters_5_kernels_predictions.npy
3/3 [==============================] - 0s 2ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_32_filters_3_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_32_filters_3_kernels_predictions.npy
3/3 [==============================] - 0s 3ms/step
Predictions saved to CSV file: magnesium/volza/predictions/100/CNN_64_filters_7_kernels_predictions.csv
Predictions saved to NPY file: magnesium/volza/predictions/100/CNN_6

In [ ]:
#Volza Only Features
aggregated_df = data.copy()
VOLZA_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN]
X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

In [ ]:
#Volza and AIS Features
aggregated_df = data.copy()
VOLZA_AIS_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN ]
aggregated_df[VOLZA_AIS_FEATURES] = aggregated_df[VOLZA_AIS_FEATURES].fillna(0)

X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_ais.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

In [ ]:
#AIS Features
aggregated_df = data.copy()
AIS_FEATURES = [SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN]
# aggregated_df = aggregated_df.dropna(subset=[SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN])
aggregated_df[AIS_FEATURES] = aggregated_df[AIS_FEATURES].fillna(method=FILL_METHOD)
X, y = data_processing.prepare_features_and_target(aggregated_df, AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

# print(y_test)
output_file_path = f'{COMMODITY}_results_ais_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

In [ ]:
#AIS and Volza with Price ARIMA
import pmdarima as pm

aggregated_df = data.copy()

columns_of_interest = ['Price', 'spikes']  # Add other columns as necessary

aggregated_df = aggregated_df.dropna(subset=columns_of_interest)
# Fit an Auto ARIMA model to the 'Price' series
model = pm.auto_arima(aggregated_df['Price'], seasonal=True, m=12, suppress_warnings=True, stepwise=True, error_action='ignore')

# Forecast the series using the model (in-sample prediction)
forecast = model.predict_in_sample()

# Calculate residuals (difference between actual and forecasted values)
residuals = aggregated_df[TARGET_COLUMN] - forecast

# Append residuals to DataFrame as a new feature (using residuals as a way to detect spike / anomaly)
aggregated_df = aggregated_df.copy()
aggregated_df['ARIMA_Residuals'] = residuals

# Prepare features and target
FEATURE_COLUMNS = [TARGET_COLUMN,VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN, 'ARIMA_Residuals']  # Adjust as needed
aggregated_df[FEATURE_COLUMNS] = aggregated_df[FEATURE_COLUMNS].fillna(0)
X, y = data_processing.prepare_features_and_target(aggregated_df, FEATURE_COLUMNS, 'spikes')

# Split data (using sklearn's train_test_split)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
X_train_scaled, X_test_scaled = data_processing.scale_features(X_train_raw, X_test_raw)

# Create sequences
X_train, y_train = data_processing.create_sequences(X_train_scaled, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test_scaled, y_test, SPIKES_WINDOW_SIZE)

# Evaluate all models & save in file
output_file_path = f'{COMMODITY}_results_volza_ais_arima.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

In [ ]:
#Binarise the data
def discretize(df, columns, bins):
    est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='kmeans')
    df[columns] = est.fit_transform(df[columns])
    return df

aggregated_df = data.copy()
VOLZA_AIS_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN ]
aggregated_df[VOLZA_AIS_FEATURES] = aggregated_df[VOLZA_AIS_FEATURES].fillna(0)
binary_columns = discretize(aggregated_df, VOLZA_AIS_FEATURES, 2)

X, y = data_processing.prepare_features_and_target(binary_columns, VOLZA_AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_ais_binary_med.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

# Try Spectral Residuals